In [1]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .master("local[*]") \
    .appName('testrdd2024') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/04/10 18:38:21 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/04/10 18:38:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


spark.stop()

In [2]:
# Taking both 2020 and 2021 data causing crash due to memory issues
df_green = spark \
            .read \
            .parquet("data/processed/green/2020/*")

In [3]:
df_green.show(5)

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2020-01-23 13:10:15|  2020-01-23 13:38:16|                 N|         1|          74|         130|              1|        12.77|       36.0|  0.0|    0.

In [4]:
df_green.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- trip_type: integer (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [5]:
df_green.rdd.take(5)

[Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), lpep_dropoff_datetime=datetime.datetime(2020, 1, 23, 13, 38, 16), store_and_fwd_flag='N', RatecodeID=1, PULocationID=74, DOLocationID=130, passenger_count=1, trip_distance=12.77, fare_amount=36.0, extra=0.0, mta_tax=0.5, tip_amount=2.05, tolls_amount=6.12, ehail_fee=None, improvement_surcharge=0.3, total_amount=44.97, payment_type=1, trip_type=1, congestion_surcharge=0.0),
 Row(VendorID=None, lpep_pickup_datetime=datetime.datetime(2020, 1, 20, 15, 9), lpep_dropoff_datetime=datetime.datetime(2020, 1, 20, 15, 46), store_and_fwd_flag=None, RatecodeID=None, PULocationID=67, DOLocationID=39, passenger_count=None, trip_distance=8.0, fare_amount=29.9, extra=2.75, mta_tax=0.5, tip_amount=0.0, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=33.45, payment_type=None, trip_type=None, congestion_surcharge=None),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 15, 20, 23, 41)

In [6]:
df_green_rdd_inter = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [7]:
df_green_rdd_inter.take(5)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), PULocationID=74, total_amount=44.97),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 20, 15, 9), PULocationID=67, total_amount=33.45),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 15, 20, 23, 41), PULocationID=260, total_amount=8.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 5, 16, 32, 26), PULocationID=82, total_amount=8.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 29, 19, 22, 42), PULocationID=166, total_amount=12.74)]

In [8]:
df_green.take(5)

[Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), lpep_dropoff_datetime=datetime.datetime(2020, 1, 23, 13, 38, 16), store_and_fwd_flag='N', RatecodeID=1, PULocationID=74, DOLocationID=130, passenger_count=1, trip_distance=12.77, fare_amount=36.0, extra=0.0, mta_tax=0.5, tip_amount=2.05, tolls_amount=6.12, ehail_fee=None, improvement_surcharge=0.3, total_amount=44.97, payment_type=1, trip_type=1, congestion_surcharge=0.0),
 Row(VendorID=None, lpep_pickup_datetime=datetime.datetime(2020, 1, 20, 15, 9), lpep_dropoff_datetime=datetime.datetime(2020, 1, 20, 15, 46), store_and_fwd_flag=None, RatecodeID=None, PULocationID=67, DOLocationID=39, passenger_count=None, trip_distance=8.0, fare_amount=29.9, extra=2.75, mta_tax=0.5, tip_amount=0.0, tolls_amount=0.0, ehail_fee=None, improvement_surcharge=0.3, total_amount=33.45, payment_type=None, trip_type=None, congestion_surcharge=None),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 15, 20, 23, 41)

'''
                        SELECT 
                              date_trunc("hour", pickup_datetime) as hour, 
                              PULocationID,
                              SUM(total_amount) as amount,
                              COUNT(1) as record_number
                        FROM green_taxi_trips
                        where pickup_datetime >= "2020-01-01 00:00:00"
                        GROUP BY 1, 2
                        --ORDER BY 1, 2
                        '''

In [9]:
df_green_rdd_inter.filter(lambda row: True).take(5)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), PULocationID=74, total_amount=44.97),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 20, 15, 9), PULocationID=67, total_amount=33.45),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 15, 20, 23, 41), PULocationID=260, total_amount=8.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 5, 16, 32, 26), PULocationID=82, total_amount=8.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 29, 19, 22, 42), PULocationID=166, total_amount=12.74)]

In [10]:
df_green_rdd_inter.filter(lambda row: False if row["total_amount"] == "BINARY" else True).take(10)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), PULocationID=74, total_amount=44.97),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 20, 15, 9), PULocationID=67, total_amount=33.45),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 15, 20, 23, 41), PULocationID=260, total_amount=8.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 5, 16, 32, 26), PULocationID=82, total_amount=8.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 29, 19, 22, 42), PULocationID=166, total_amount=12.74),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 15, 11, 7, 42), PULocationID=179, total_amount=5.8),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 16, 8, 22, 29), PULocationID=41, total_amount=25.05),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 28, 17, 5, 28), PULocationID=75, total_amount=21.33),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 22, 14, 51, 37), PULocationID=152, total_amount=7.8),
 Row(lpep_pickup_datetime=datetime.dateti

In [11]:
df_green_rdd_inter.filter(lambda row: True if isinstance(row["total_amount"], float) else False).take(1)


[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), PULocationID=74, total_amount=44.97)]

In [12]:
df_green_rdd = df_green_rdd_inter \
                .filter(lambda row: False if row["total_amount"] == "BINARY" else True)

In [13]:
df_green_rdd.take(5)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), PULocationID=74, total_amount=44.97),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 20, 15, 9), PULocationID=67, total_amount=33.45),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 15, 20, 23, 41), PULocationID=260, total_amount=8.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 5, 16, 32, 26), PULocationID=82, total_amount=8.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 29, 19, 22, 42), PULocationID=166, total_amount=12.74)]

In [14]:
from datetime import datetime

In [15]:
start = datetime(2020, 1, 1)

In [16]:
df_green_rdd \
        .filter(lambda row: row["lpep_pickup_datetime"] >= start).take(5)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), PULocationID=74, total_amount=44.97),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 20, 15, 9), PULocationID=67, total_amount=33.45),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 15, 20, 23, 41), PULocationID=260, total_amount=8.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 5, 16, 32, 26), PULocationID=82, total_amount=8.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 29, 19, 22, 42), PULocationID=166, total_amount=12.74)]

In [17]:
def filter_by_date(row):
    start = datetime(2020, 1, 1)
    return row['lpep_pickup_datetime'] >= start

In [18]:
df_green_row = df_green_rdd.take(5)[0]
df_green_row

Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), PULocationID=74, total_amount=44.97)

In [19]:
filter_by_date(df_green_row)

True

In [20]:
def prepare_for_grouping(row):
    hour = row['lpep_pickup_datetime'].replace(minute=0, second=0)
    zone = row['PULocationID']
    amount = row['total_amount']
    count = 1
    key = (hour, zone)
    value = (amount, count)
    return (key, value)

In [21]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value
    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    return (output_amount, output_count)

In [22]:
df_green_rdd \
                .filter(filter_by_date) \
                .map(prepare_for_grouping) \
                .take(5)

[((datetime.datetime(2020, 1, 23, 13, 0), 74), (44.97, 1)),
 ((datetime.datetime(2020, 1, 20, 15, 0), 67), (33.45, 1)),
 ((datetime.datetime(2020, 1, 15, 20, 0), 260), (8.3, 1)),
 ((datetime.datetime(2020, 1, 5, 16, 0), 82), (8.3, 1)),
 ((datetime.datetime(2020, 1, 29, 19, 0), 166), (12.74, 1))]

In [23]:
df_green_rdd \
    .filter(filter_by_date) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .take(5)



[((datetime.datetime(2020, 1, 20, 15, 0), 67), (79.5, 3)),
 ((datetime.datetime(2020, 1, 5, 16, 0), 82), (500.4700000000002, 33)),
 ((datetime.datetime(2020, 1, 16, 8, 0), 41), (736.1399999999996, 54)),
 ((datetime.datetime(2020, 1, 20, 15, 0), 75), (608.9999999999999, 47)),
 ((datetime.datetime(2020, 1, 17, 21, 0), 74), (594.87, 39))]

In [24]:
def ungroup(row):
    key, value = row
    hour, zone = key
    amount, count = value
    return (hour, zone, amount, count)

In [25]:
df_green_rdd \
    .filter(filter_by_date) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(ungroup) \
    .take(5)

[(datetime.datetime(2020, 1, 20, 15, 0), 67, 79.5, 3),
 (datetime.datetime(2020, 1, 5, 16, 0), 82, 500.4700000000002, 33),
 (datetime.datetime(2020, 1, 16, 8, 0), 41, 736.1399999999996, 54),
 (datetime.datetime(2020, 1, 20, 15, 0), 75, 608.9999999999999, 47),
 (datetime.datetime(2020, 1, 17, 21, 0), 74, 594.87, 39)]

In [26]:
df_green_rdd \
    .filter(filter_by_date) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(ungroup) \
    .toDF(["hour", "zone", "revenue", "count"]) \
    .show(5)

+-------------------+----+-----------------+-----+
|               hour|zone|          revenue|count|
+-------------------+----+-----------------+-----+
|2020-01-20 15:00:00|  67|             79.5|    3|
|2020-01-05 16:00:00|  82|500.4700000000002|   33|
|2020-01-16 08:00:00|  41|736.1399999999996|   54|
|2020-01-20 15:00:00|  75|608.9999999999999|   47|
|2020-01-17 21:00:00|  74|           594.87|   39|
+-------------------+----+-----------------+-----+
only showing top 5 rows



In [27]:
df_green_rdd_result = df_green_rdd \
    .filter(filter_by_date) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(ungroup) \
    .toDF(["hour", "zone", "revenue", "count"])

In [28]:
df_green_rdd_result.printSchema()

root
 |-- hour: timestamp (nullable = true)
 |-- zone: long (nullable = true)
 |-- revenue: double (nullable = true)
 |-- count: long (nullable = true)



In [55]:
from pyspark.sql.types import StructType, StructField, TimestampType, IntegerType, FloatType, StringType

gree_rdd_result_schema = StructType([
    StructField("hour", TimestampType(), True),
    StructField("zone", IntegerType(), True),
    StructField("revenue", FloatType(), True),
    StructField("count", IntegerType(), True)
])

In [30]:
df_green_rdd_result = df_green_rdd \
    .filter(filter_by_date) \
    .map(prepare_for_grouping) \
    .reduceByKey(calculate_revenue) \
    .map(ungroup) \
    .toDF(gree_rdd_result_schema)

In [31]:
df_green_rdd_result.show(5)

+-------------------+----+-------+-----+
|               hour|zone|revenue|count|
+-------------------+----+-------+-----+
|2020-01-20 15:00:00|  67|   79.5|    3|
|2020-01-05 16:00:00|  82| 500.47|   33|
|2020-01-16 08:00:00|  41| 736.14|   54|
|2020-01-20 15:00:00|  75|  609.0|   47|
|2020-01-17 21:00:00|  74| 594.87|   39|
+-------------------+----+-------+-----+
only showing top 5 rows



In [98]:
df_green_rdd_result.write \
    .mode("overwrite") \
    .parquet("data/processed/green/rdd/2020_revenue")

In [32]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']

green_duration_rdd = df_green \
    .select(columns) \
    .rdd

In [33]:
def apply_model_in_batch(rows):
    return [1]

In [34]:
green_duration_rdd.take(5) 

[Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), PULocationID=74, DOLocationID=130, trip_distance=12.77),
 Row(VendorID=None, lpep_pickup_datetime=datetime.datetime(2020, 1, 20, 15, 9), PULocationID=67, DOLocationID=39, trip_distance=8.0),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 15, 20, 23, 41), PULocationID=260, DOLocationID=157, trip_distance=1.27),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 5, 16, 32, 26), PULocationID=82, DOLocationID=83, trip_distance=1.25),
 Row(VendorID=2, lpep_pickup_datetime=datetime.datetime(2020, 1, 29, 19, 22, 42), PULocationID=166, DOLocationID=42, trip_distance=1.84)]

In [35]:
green_duration_rdd.mapPartitions(apply_model_in_batch).collect()


[1, 1, 1, 1]

In [36]:
def apply_model_in_batch(partition):
    cnt = 0
    for row in partition:
        cnt += 1
    yield cnt

In [37]:
green_duration_rdd.mapPartitions(apply_model_in_batch).collect()


[902254, 395273, 280443, 156081]

In [38]:
rows = green_duration_rdd.take(5)

In [45]:
# defining a  genarator
def infinite_seq():
    i = 0
    while True:
        yield i
        i += 1
# seq is a generator object below
seq = infinite_seq()
print(seq)

<generator object infinite_seq at 0x7fe7eb1fcc80>


In [48]:
# materializing seq will hang as it contains infinite sequence instead we can use next to get the next value
next(seq)

2

In [49]:
for i in range(5):
    print(next(seq))

3
4
5
6
7


In [43]:
import pandas as pd
df_iter_test = pd.DataFrame(rows, columns=columns)
# Need materializing itert
list(df_iter_test.itertuples())

[Pandas(Index=0, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-23 13:10:15'), PULocationID=74, DOLocationID=130, trip_distance=12.77),
 Pandas(Index=1, VendorID=nan, lpep_pickup_datetime=Timestamp('2020-01-20 15:09:00'), PULocationID=67, DOLocationID=39, trip_distance=8.0),
 Pandas(Index=2, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-15 20:23:41'), PULocationID=260, DOLocationID=157, trip_distance=1.27),
 Pandas(Index=3, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-05 16:32:26'), PULocationID=82, DOLocationID=83, trip_distance=1.25),
 Pandas(Index=4, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-29 19:22:42'), PULocationID=166, DOLocationID=42, trip_distance=1.84)]

In [40]:
def apply_model_in_batch(partition):
    df_mappart = pd.DataFrame(partition, columns=columns)
    model
    cnt = len(df_mappart)
    return [cnt]

In [131]:
green_duration_rdd.mapPartitions(apply_model_in_batch).collect()

[902254, 395273, 280443, 156081]

In [127]:
# investigate how to use itertools for larger chunk of partitions in above example

In [41]:
def model_predict(df):
    df['trip_duration'] =   df.trip_distance * 2
    return df
    

In [75]:
def apply_model_in_batch(partition):
    df_mappart = pd.DataFrame(partition, columns=columns)
    model_predict(df_mappart)
    cnt = len(df_mappart)
    for row in df_mappart.itertuples(index=True, name='Pandas'):
        yield row

In [76]:
green_duration_rdd.mapPartitions(apply_model_in_batch).take(5)

[Pandas(Index=0, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-23 13:10:15'), PULocationID=74, DOLocationID=130, trip_distance=12.77, trip_duration=25.54),
 Pandas(Index=1, VendorID=nan, lpep_pickup_datetime=Timestamp('2020-01-20 15:09:00'), PULocationID=67, DOLocationID=39, trip_distance=8.0, trip_duration=16.0),
 Pandas(Index=2, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-15 20:23:41'), PULocationID=260, DOLocationID=157, trip_distance=1.27, trip_duration=2.54),
 Pandas(Index=3, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-05 16:32:26'), PULocationID=82, DOLocationID=83, trip_distance=1.25, trip_duration=2.5),
 Pandas(Index=4, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-29 19:22:42'), PULocationID=166, DOLocationID=42, trip_distance=1.84, trip_duration=3.68)]

In [77]:
model_output_schema = StructType([
    StructField('Row_index', IntegerType(), True),
    StructField('VendorID', StringType(), True),
    StructField('lpep_pickup_datetime', TimestampType(), True),
    StructField('PULocationID', IntegerType(), True),
    StructField('DOLocationID', IntegerType(), True),
    StructField('trip_distance', FloatType(), True),
    StructField('trip_duration', FloatType(), True)
])

In [78]:
green_duration_rdd \
    .mapPartitions(apply_model_in_batch) \
    .take(5)

[Pandas(Index=0, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-23 13:10:15'), PULocationID=74, DOLocationID=130, trip_distance=12.77, trip_duration=25.54),
 Pandas(Index=1, VendorID=nan, lpep_pickup_datetime=Timestamp('2020-01-20 15:09:00'), PULocationID=67, DOLocationID=39, trip_distance=8.0, trip_duration=16.0),
 Pandas(Index=2, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-15 20:23:41'), PULocationID=260, DOLocationID=157, trip_distance=1.27, trip_duration=2.54),
 Pandas(Index=3, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-05 16:32:26'), PULocationID=82, DOLocationID=83, trip_distance=1.25, trip_duration=2.5),
 Pandas(Index=4, VendorID=2.0, lpep_pickup_datetime=Timestamp('2020-01-29 19:22:42'), PULocationID=166, DOLocationID=42, trip_distance=1.84, trip_duration=3.68)]

In [80]:
green_duration_rdd.mapPartitions(apply_model_in_batch).toDF().show(5)

+-----+--------+--------------------+------------+------------+-------------+-------------+
|Index|VendorID|lpep_pickup_datetime|PULocationID|DOLocationID|trip_distance|trip_duration|
+-----+--------+--------------------+------------+------------+-------------+-------------+
|    0|     2.0|                  {}|          74|         130|        12.77|        25.54|
|    1|     NaN|                  {}|          67|          39|          8.0|         16.0|
|    2|     2.0|                  {}|         260|         157|         1.27|         2.54|
|    3|     2.0|                  {}|          82|          83|         1.25|          2.5|
|    4|     2.0|                  {}|         166|          42|         1.84|         3.68|
+-----+--------+--------------------+------------+------------+-------------+-------------+
only showing top 5 rows



In [81]:
df_green_duration = green_duration_rdd \
    .mapPartitions(apply_model_in_batch) \
    .toDF()

In [82]:
df_green_duration \
    .select('trip_distance', 'trip_duration') \
    .show(5)

+-------------+-------------+
|trip_distance|trip_duration|
+-------------+-------------+
|        12.77|        25.54|
|          8.0|         16.0|
|         1.27|         2.54|
|         1.25|          2.5|
|         1.84|         3.68|
+-------------+-------------+
only showing top 5 rows



In [83]:
spark.stop()